In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load and preprocess the dataset
data = pd.read_csv('MLTempDataset1.csv') 
data['Datetime'] = pd.to_datetime(data['Datetime'])
data.set_index('Datetime', inplace=True)

# Extract and normalize temperature values
values = data['Hourly_Temp'].values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_values = scaler.fit_transform(values.reshape(-1, 1))

# Create sequences for RNN
def create_dataset(series, time_step=1):
    X, y = [], []
    for i in range(len(series) - time_step):
        X.append(series[i:(i + time_step), 0])
        y.append(series[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 10
X, y = create_dataset(scaled_values, time_step)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Build and compile the RNN model
model = Sequential()
model.add(SimpleRNN(50, input_shape=(time_step, 1)))
model.add(Dense(1))  # No activation function for regression
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")

# Make predictions and invert scaling
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Display sample predictions and true values
print("Sample Predictions:", predictions[:5].flatten())
print("Sample True Values:", y_test_inv[:5].flatten())


Epoch 1/10


c:\Users\TEST\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.0491 - val_loss: 0.0092
Epoch 2/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0130 - val_loss: 0.0072
Epoch 3/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0101 - val_loss: 0.0061
Epoch 4/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0093 - val_loss: 0.0061
Epoch 5/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0107 - val_loss: 0.0058
Epoch 6/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0096 - val_loss: 0.0061
Epoch 7/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0094 - val_loss: 0.0059
Epoch 8/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0093 - val_loss: 0.0058
Epoch 9/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0088 - val_loss: 0.0058
Epoch 10/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0093 - val_loss: 0.0059
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0028   
Test Loss: 0.0031
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Sample Predictions: [25.530636 25.20

In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load and preprocess the dataset
data = pd.read_csv('MLTempDataset1.csv')  # Replace with your CSV file path
data['Datetime'] = pd.to_datetime(data['Datetime'])
data.set_index('Datetime', inplace=True)

# Extract and normalize temperature values
values = data['Hourly_Temp'].values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_values = scaler.fit_transform(values.reshape(-1, 1))

# Create sequences for RNN
def create_dataset(series, time_step=1):
    X, y = [], []
    for i in range(len(series) - time_step):
        X.append(series[i:(i + time_step), 0])
        y.append(series[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 10
X, y = create_dataset(scaled_values, time_step)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Build and train stacked RNN model
model_stacked = Sequential()
model_stacked.add(SimpleRNN(50, return_sequences=True, input_shape=(time_step, 1)))
model_stacked.add(SimpleRNN(30))
model_stacked.add(Dense(1))  # For regression tasks
model_stacked.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model_stacked.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate stacked RNN model
loss_stacked, mae_stacked = model_stacked.evaluate(X_test, y_test)
print(f"Stacked RNN Test Loss: {loss_stacked:.4f}")
print(f"Stacked RNN Test MAE: {mae_stacked:.4f}")

# Build and train bi-directional RNN model
model_bidirectional = Sequential()
model_bidirectional.add(Bidirectional(SimpleRNN(50, return_sequences=True), input_shape=(time_step, 1)))
model_bidirectional.add(SimpleRNN(30))
model_bidirectional.add(Dense(1))  # For regression tasks
model_bidirectional.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model_bidirectional.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate bi-directional RNN model
loss_bidirectional, mae_bidirectional = model_bidirectional.evaluate(X_test, y_test)
print(f"Bi-directional RNN Test Loss: {loss_bidirectional:.4f}")
print(f"Bi-directional RNN Test MAE: {mae_bidirectional:.4f}")

# Make and display predictions for both models
predictions_stacked = model_stacked.predict(X_test)
predictions_bidirectional = model_bidirectional.predict(X_test)

# Inverse transform the predictions and actual values
predictions_stacked = scaler.inverse_transform(predictions_stacked)
predictions_bidirectional = scaler.inverse_transform(predictions_bidirectional)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

print("Stacked RNN Sample Predictions:", predictions_stacked[:5].flatten())
print("Stacked RNN Sample True Values:", y_test_inv[:5].flatten())
print("Bi-directional RNN Sample Predictions:", predictions_bidirectional[:5].flatten())
print("Bi-directional RNN Sample True Values:", y_test_inv[:5].flatten())


Epoch 1/10


c:\Users\TEST\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


141/141 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - loss: 0.0515 - mae: 0.1542 - val_loss: 0.0078 - val_mae: 0.0619
Epoch 2/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0114 - mae: 0.0678 - val_loss: 0.0059 - val_mae: 0.0393
Epoch 3/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0099 - mae: 0.0593 - val_loss: 0.0063 - val_mae: 0.0507
Epoch 4/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0095 - mae: 0.0590 - val_loss: 0.0084 - val_mae: 0.0611
Epoch 5/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0103 - mae: 0.0616 - val_loss: 0.0065 - val_mae: 0.0423
Epoch 6/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0099 - mae: 0.0589 - val_loss: 0.0072 - val_mae: 0.0632
Epoch 7/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0098 - mae: 0.0607 - val_loss: 0.0068 - val_mae: 0.0462
Epoch 8/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0105 - mae: 0.0656 - val_loss: 0.0059 - val_mae: 0.0362
Epoch 9/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 

c:\Users\TEST\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


141/141 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - loss: 0.0500 - mae: 0.1442 - val_loss: 0.0059 - val_mae: 0.0388
Epoch 2/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0094 - mae: 0.0601 - val_loss: 0.0061 - val_mae: 0.0470
Epoch 3/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0106 - mae: 0.0614 - val_loss: 0.0063 - val_mae: 0.0540
Epoch 4/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0095 - mae: 0.0615 - val_loss: 0.0064 - val_mae: 0.0434
Epoch 5/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0100 - mae: 0.0606 - val_loss: 0.0059 - val_mae: 0.0361
Epoch 6/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0099 - mae: 0.0594 - val_loss: 0.0063 - val_mae: 0.0420
Epoch 7/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0098 - mae: 0.0623 - val_loss: 0.0061 - val_mae: 0.0373
Epoch 8/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0091 - mae: 0.0582 - val_loss: 0.0059 - val_mae: 0.0429
Epoch 9/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss

In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load and preprocess the dataset
data = pd.read_csv('MLTempDataset1.csv')  # Replace with your CSV file path
data['Datetime'] = pd.to_datetime(data['Datetime'])
data.set_index('Datetime', inplace=True)

# Extract and normalize temperature values
values = data['Hourly_Temp'].values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_values = scaler.fit_transform(values.reshape(-1, 1))

# Create sequences for the model
def create_dataset(series, time_step=1):
    X, y = [], []
    for i in range(len(series) - time_step):
        X.append(series[i:(i + time_step), 0])
        y.append(series[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 10
X, y = create_dataset(scaled_values, time_step)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Build Hybrid Model (CNN + Dense)
model_hybrid = Sequential()
model_hybrid.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(time_step, 1)))
model_hybrid.add(MaxPooling1D(pool_size=2))
model_hybrid.add(Flatten())
model_hybrid.add(Dense(50, activation='relu'))
model_hybrid.add(Dense(1))  # For regression tasks (use 'sigmoid' for binary classification)
model_hybrid.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train hybrid model
model_hybrid.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate hybrid model
loss_hybrid, mae_hybrid = model_hybrid.evaluate(X_test, y_test)
print(f"Hybrid Model Test Loss: {loss_hybrid:.4f}")
print(f"Hybrid Model Test MAE: {mae_hybrid:.4f}")

# Make predictions
predictions_hybrid = model_hybrid.predict(X_test)

# Inverse transform the predictions and actual values
predictions_hybrid = scaler.inverse_transform(predictions_hybrid)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Display results
print("Hybrid Model Sample Predictions:", predictions_hybrid[:5].flatten())
print("Hybrid Model Sample True Values:", y_test_inv[:5].flatten())


c:\Users\TEST\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.0705 - mae: 0.1827 - val_loss: 0.0091 - val_mae: 0.0666
Epoch 2/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0122 - mae: 0.0713 - val_loss: 0.0072 - val_mae: 0.0421
Epoch 3/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0105 - mae: 0.0641 - val_loss: 0.0073 - val_mae: 0.0449
Epoch 4/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0120 - mae: 0.0701 - val_loss: 0.0068 - val_mae: 0.0497
Epoch 5/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0105 - mae: 0.0622 - val_loss: 0.0065 - val_mae: 0.0434
Epoch 6/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0096 - mae: 0.0573 - val_loss: 0.0080 - val_mae: 0.0560
Epoch 7/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0103 - mae: 0.0632 - val_loss: 0.0075 - val_mae: 0.0516
Epoch 8/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0100 - mae: 0.0597 - val_loss: 0.0063 - val_mae: 0.0388
Epoch 9/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - l